In [86]:
pip install pyenchant 


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [87]:
import nltk   #importing libraries
import re
import os
import bisect
import os,glob
import pandas as pd  
import enchant

In [88]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/nitish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nitish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nitish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [89]:
documentsName=os.listdir("/home/nitish/Downloads/stories")   #titles of the files
all_files = glob.glob(os.path.join("/home/nitish/Downloads/stories", "*"))     # location of the files.....os.path.join  makes concatenation OS independent


In [90]:
def remove_consecutive_dups(s):   #removing unnecessary letters from extended words
    return re.sub(r'(?i)(.)\1+', r'\1', s)

In [91]:
check_eng = enchant.Dict("en_US")   # using en_US to see valid english words
def Preprocess(text):       
    text = re.sub('[^A-Za-z ]+', ' ',text)   # keeping only alphabetical words
    words = word_tokenize(text)      # tokenize
    tokens = []
    for word in words:
        if word not in set(stopwords.words('english')):    #removal stopword
                if re.findall("([a-zA-Z])\\1{3,}",word):   #checking elongated words.....more than 2 repeated letters
                    word = remove_consecutive_dups(word)
                if (check_eng.check(word)):                    #is valid english word
                    word = word.lower()
                    tokens.append(word)    
    return tokens

In [95]:

documentID={}         # for DocID, keys will be docNumber
invertedIndex = {}    #dictionary for inverted index, keys will be tokens

for j in range(len(all_files)):       #will open all files one by one
    documentText = ""
    column=['text']                     #column name in dataframe will be "text"
    df = pd.read_csv(all_files[j], names=column, sep="\t,\n, ")     #load file in a dataframe
    for i in df['text']:
        documentText = documentText+i+" "
    
    tokens = set(preprocess(documentText))    # remove duplicate
    for token in tokens:                      
        if token not in invertedIndex:
            posting = []             #if token not already in invertedIndex creating a new entry
            posting.append(j+1)
            invertedIndex[token] = posting
        else:
            posting = invertedIndex[token]    #updating docID
            bisect.insort(posting, j+1)
            invertedIndex[token] = posting
    documentID[j+1]=documentsName[j]
        

<ipython-input-95-3e2ac2e2721d>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(all_files[j], names=column, sep="\t,\n, ")     #load file in a dataframe


In [96]:
print(len(invertedIndex))

47592


In [97]:
for key in invertedIndex.keys():            #putting number od docs as first entry in posting of every token
    invertedIndex[key].insert(0,len(invertedIndex[key]))    
#print(invertedIndex)


In [98]:
def AND(opearand1,opearnd2):
    result = []
    i = 1
    j =1
    comparisons = 0
    postingList1= invertedIndex[opearand1]
    postingList2= invertedIndex[opearand2]
    while(i<=postingList1[0] and j<=postingList2[0]):
        comparisons= comparisons+1
        if(postingList1[i] == postingList2[j]):
            result.append(postingList1[i])
            i=i+1
            j=j+1
        elif(postingList1[i] < postingList2[j]):
            i=i+1
        else:
            j=j+1
    while(i<=postingList1[0]):
        result.append(postingList1[i])  
        i = i+1
    while(j<=postingList2[0]):
        result.append(postingList2[j])  
        j = j+1
        
    result.insert(0,len(result))       
  
    return comparisons,result

In [99]:
def OR(postingList1,postingList2):
    result = []
    i = 1
    j =1
    comparisons = 0
    while(i<=postingList1[0] and j<=postingList2[0]):
        comparisons= comparisons+1
        if(postingList1[i] == postingList2[j]):
            result.append(postingList1[i])
            i=i+1
            j=j+1
        elif(postingList1[i] < postingList2[j]):
            result.append(postingList1[i])
            i=i+1
        else:
            result.append(postingList2[j])
            j=j+1
            
            
    while(i<=postingList1[0]):
        result.append(postingList1[i]) 
        i=i+1
    while(j<=postingList2[0]):
        result.append(postingList2[j])  
        j=j+1
        
    result.insert(0,len(result))       
  
    return comparisons,result

In [100]:
def AND_NOT(postingList1,postingList2):
    result = []
    i = 1
    j =1
    comparisons = 0
    while(i<=postingList1[0] and j<=postingList2[0]):
        comparisons= comparisons+1
        if(postingList1[i] == postingList2[j]):
            i=i+1
            j=j+1
        elif(postingList1[i] < postingList2[j]):
            result.append(postingList1[i])
            i=i+1
        else:
            j=j+1
            
            
    while(i<=postingList1[0]):
        result.append(postingList1[i]) 
        i=i+1
        
    result.insert(0,len(result))       
  
    return comparisons,result

In [101]:
def OR_NOT(postingList1,postingList2):
    # total doc AND_NOT {B AND_NOT A}
    TotalDoc= []
    comparisons = 0
    for i in range(1,468):
        TotalDoc.append(i)
    TotalDoc.insert(0,len(TotalDoc))       
    com,result = AND_NOT(postingList2,postingList1)
    comparisons = com
    com,result = AND_NOT(TotalDoc,result)
    comparisons = comparisons + com 
    
    return comparisons,result
    

In [102]:
def NOT(postingList2):
    TotalDoc= []
    comparisons = 0
    for i in range(1,468):
        TotalDoc.append(i)
    TotalDoc.insert(0,len(TotalDoc)) 
    result = []
    i = 1
    j =1
    while(i<=TotalDoc[0] and j<=postingList2[0]):
        comparisons= comparisons+1
        if(TotalDoc[i] == postingList2[j]):
            i=i+1
            j=j+1
        elif(TotalDoc[i] < postingList2[j]):
            result.append(TotalDoc[i])
            i=i+1
        else:
            j=j+1
            
            
    while(i<=TotalDoc[0]):
        result.append(TotalDoc[i]) 
        i=i+1
        
    result.insert(0,len(result))       
  
    return comparisons,result 

In [103]:
NoOfQueries =int(input())            

while NoOfQueries!=0 :
    query =input()
    operators =input() 
    operands = Preprocess(query)    #Preprocessing the query
    operators = operators.split(',')  # preprocessing operators
    
    operators[0]=operators[0].replace('[',"")
    operators[len(operators)-1]=operators[len(operators)-1].replace(']',"")
    for i in range(len(operators)):
        operators[i] = operators[i].strip()
    
    count = len(operators)
    result = []
    i = 0
    comparisons = 0
    while(i<count):
        if(i==0):
            list1 = invertedIndex[operands[0]]
        else:
            list1 = result
        list2 = invertedIndex[operands[i+1]]
        if(operators[i]=='AND'):
            com,result = AND(list1,list2) 
        elif(operators[i]=='OR'):
            
            com,result = OR(list1,list2) 
        elif(operators[i]=='AND NOT'):
            com,result = AND_NOT(list1,list2)
        else:
            com,result = OR_NOT(list1,list2)
            
            
        comparisons = comparisons + com
        i = i+1

    
    
    print("Number of documents matched: ",result[0])
    print("No. of comparisons required: ",comparisons)
    retrivedDocuments = []
    for i in result:
        retrivedDocuments.append(documentID[i])
    #print("list of document names retrieved: ",retrivedDocuments)
    NoOfQueries = NoOfQueries-1
    
    
    

 2
 lion stood thoughtfully for a moment
 [OR,OR,OR]


Number of documents matched:  265
No. of comparisons required:  663


 telephone,paved, roads
 [OR NOT, AND NOT]


Number of documents matched:  427
No. of comparisons required:  814
